In [1]:
# Install this libraries
# !pip install torch
# !pip install torch-geometric
# !pip install torch_spline_conv
# !pip install torch_scatter 
# !pip install torch_cluster #slow
# !pip install torch_sparse #slow
# !pip install matplotlib
# !pip install ogb
# !pip install networkx

In [1]:
from __future__ import division
from __future__ import print_function

import time
import argparse
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim
import random
from models import GCN

import torch_geometric
import torch_sparse

import matplotlib.pyplot as plt
from sklearn import metrics

from utils import get_plot


In [2]:
from data_process import generate_data, load_data, load_data_for_amazon, load_data_fraudre
from train_func import eval_metrics, test, train, Lhop_Block_matrix_train, FedSage_train

In [3]:
def get_K_hop_neighbors(adj_matrix, index, K):
    adj_matrix = adj_matrix + torch.eye(adj_matrix.shape[0],adj_matrix.shape[1])  #make sure the diagonal part >= 1
    hop_neightbor_index=index
    for i in range(K):
        hop_neightbor_index=torch.unique(torch.nonzero(adj[hop_neightbor_index])[:,1])
    return hop_neightbor_index

In [4]:
import scipy.sparse as sp
def normalize(mx):  #adj matrix
    
    mx = mx + torch.eye(mx.shape[0],mx.shape[1])
    
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return torch.tensor(mx)

In [5]:
def setdiff1d(t1, t2):
    
    combined = torch.cat((t1, t2))
    uniques, counts = combined.unique(return_counts=True)
    difference = uniques[counts == 1]
    #intersection = uniques[counts > 1]
    return difference

In [6]:
def intersect1d(t1, t2):
    
    combined = torch.cat((t1, t2))
    uniques, counts = combined.unique(return_counts=True)
    #difference = uniques[counts == 1]
    intersection = uniques[counts > 1]
    return intersection

# Model

In [7]:
#define model

#for compare 2-10 layer performance in appendix
#iterations = 400
#Adam, lr = 0.01


def centralized_GCN(features, adj, labels, idx_train, idx_val, idx_test, num_layers):
    
        model = GCN(nfeat=features.shape[1],
                nhid=args_hidden,
                nclass=labels.max().item() + 1,
                dropout=args_dropout,
                NumLayers=num_layers)
        model.reset_parameters()
        if args_cuda:
                #from torch_geometric.nn import DataParallel
                #model = DataParallel(model)
                #model= torch.nn.DataParallel(model)
                model=model#.cuda()
                
                #features= torch.nn.DataParallel(features)
                
                features = features#.cuda()
                
                #edge_index= torch.nn.DataParallel(edge_index)
                
                adj = adj#.cuda()
                labels = labels#.cuda()
                idx_train = idx_train#.cuda()
                idx_val = idx_val#.cuda()
                idx_test = idx_test#.cuda()
        #optimizer and train
        
        optimizer = optim.SGD(model.parameters(),
                              lr=args_lr, weight_decay=args_weight_decay)
        
        
        #optimizer = optim.Adam(model.parameters(),
        #                      lr=args_lr, weight_decay=args_weight_decay)
        # Train model
        best_val=0
        for t in range(args_iterations): #make to equivalent to federated
            loss_train, acc_train=train(t, model, optimizer, features, adj, labels, idx_train)
            # validation
            loss_train, acc_train= test(model, features, adj, labels, idx_train) #train after backward
            print(t,"train",loss_train,acc_train)
            loss_val, acc_val= test(model, features, adj, labels, idx_val) #validation
            print(t,"val",loss_val,acc_val)
            
            a = open(dataset_name+'_IID_'+'centralized_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations),'a+')
            a.write(str(t)+'\t'+"train"+'\t'+str(loss_train)+'\t'+str(acc_train)+'\n')
            a.write(str(t)+'\t'+"val"+'\t'+str(loss_val)+'\t'+str(acc_val)+'\n')
            a.close()
            
        #test  
        loss_test, acc_test= test(model, features, adj, labels, idx_test)
        print(t,'\t',"test",'\t',loss_test,'\t',acc_test)
        #evaluation metrics -> Precision, Recall, Balanced Accuracy, and F1-score. 
        precision_sc, recalL_sc, balanced_acc_sc, f1_test = eval_metrics(model, features, adj, labels, idx_test)
        print(f"{precision_sc}, {recalL_sc}, {balanced_acc_sc}, {f1_test}".format(precision_sc,recalL_sc,balanced_acc_sc,f1_test))

        a = open(dataset_name+'_IID_'+'centralized_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations),'a+')
        a.write(str(t)+'\t'+"test"+'\t'+str(loss_test)+'\t'+str(acc_test)+'\n')
        a.write(str(t)+'\t'+str(precision_sc) + '\t'+str(recalL_sc) +'\t' +str(balanced_acc_sc) +'\t' +str(f1_test))
        a.close()
        
        print("save file as",dataset_name+'_IID_'+'centralized_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations))
        del model
        del features 
        del adj
        del labels
        del idx_train
        del idx_val
        del idx_test
        
        return loss_test, acc_test

In [8]:
def BDS_federated_GCN(K, features, adj, labels, idx_train, idx_val, idx_test, iid_percent, sample_rate =0.5, L_hop=1, num_layers=2):
        # K: number of models
        K = int(K)
        #choose adj matrix
        #multilayer_GCN:n*n
        #define model
        global_model = GCN(nfeat=features.shape[1],
                nhid=args_hidden,
                nclass=labels.max().item() + 1,
                dropout=args_dropout,
                NumLayers=num_layers)
        global_model.reset_parameters()
        models=[]
        for i in range(K):
            models.append(GCN(nfeat=features.shape[1],
                nhid=args_hidden,
                nclass=labels.max().item() + 1,
                dropout=args_dropout,
                NumLayers=num_layers))
        if args_cuda:
                for i in range(K):
                    models[i]=models[i].cuda()
                global_model=global_model.cuda()
                features = features.cuda()
                adj = adj.cuda()
                labels = labels.cuda()
                idx_train = idx_train.cuda()
                idx_val = idx_val.cuda()
                idx_test = idx_test.cuda()
        #optimizer and train
        optimizers=[]
        for i in range(K):
            optimizers.append(optim.SGD(models[i].parameters(),
                              lr=args_lr, weight_decay=args_weight_decay))
        # Train model
        
        row, col, edge_attr = adj.t().coo()
        edge_index = torch.stack([row, col], dim=0)
        
        
        split_data_indexes=[]
        
        nclass=labels.max().item() + 1
        split_data_indexes = []
        non_iid_percent = 1 - float(iid_percent)
        iid_indexes = [] #random assign
        shuffle_labels = [] #make train data points split into different devices
        for i in range(K):
            current = torch.nonzero(labels == i).reshape(-1)
            current = current[np.random.permutation(len(current))] #shuffle
            shuffle_labels.append(current)
                
        average_device_of_class = K // nclass
        if K % nclass != 0: #for non-iid
            average_device_of_class += 1
        for i in range(K):  
            label_i= int(i // average_device_of_class)    
            labels_class = shuffle_labels[label_i]

            average_num= int(len(labels_class)//average_device_of_class * non_iid_percent)
            split_data_indexes.append((labels_class[int(average_num * (i % average_device_of_class)):int(average_num * (i % average_device_of_class + 1))]))
        
        if args_cuda:
            iid_indexes = setdiff1d(torch.tensor(range(len(labels))).cuda(), torch.cat(split_data_indexes))
        else:
            iid_indexes = setdiff1d(torch.tensor(range(len(labels))), torch.cat(split_data_indexes))
        
        iid_indexes = iid_indexes[np.random.permutation(len(iid_indexes))]
        
        for i in range(K):  #for iid
            label_i= int(i // average_device_of_class)
            labels_class = shuffle_labels[label_i]

            average_num= int(len(labels_class)//average_device_of_class * (1 - non_iid_percent))
            split_data_indexes[i] = list(split_data_indexes[i]) + list(iid_indexes[:average_num])
                    
            iid_indexes = iid_indexes[average_num:]
            
        communicate_indexes = []
        in_com_train_data_indexes = []

        for i in range(K):
            if args_cuda:
                split_data_indexes[i] = torch.tensor(split_data_indexes[i]).cuda()
            else:
                split_data_indexes[i] = torch.tensor(split_data_indexes[i])
                
            split_data_indexes[i] = split_data_indexes[i].sort()[0]
            
            #communicate_index=get_K_hop_neighbors(adj, split_data_indexes[i], L_hop) #normalized adj
            
            communicate_index = torch_geometric.utils.k_hop_subgraph(split_data_indexes[i],L_hop,edge_index)[0]
            communicate_indexes.append(communicate_index)
            communicate_indexes[i] = communicate_indexes[i].sort()[0]
            
            inter = intersect1d(split_data_indexes[i], idx_train)  ###only count the train data of nodes in current server(not communicate nodes)   
            in_com_train_data_indexes.append(torch.searchsorted(communicate_indexes[i], inter).clone())   #local id in block matrix

            
            
        #assign global model weights to local models at initial step
        for i in range(K):
            models[i].load_state_dict(global_model.state_dict())
        
        for t in range(args_iterations):
            acc_trains=[]
            for i in range(K):
                for epoch in range(args_epochs):
                    diff = setdiff1d(split_data_indexes[i], communicate_indexes[i])
                    sample_index = torch.cat((split_data_indexes[i], diff[torch.randperm(len(diff))[:int(len(diff) * sample_rate)]])).clone()

                    sample_index = sample_index.sort()[0]
                    
                    inter = intersect1d(split_data_indexes[i], idx_train)  ###only count the train data of nodes in current server(not communicate nodes)
                    in_sample_train_data_index = torch.searchsorted(sample_index, inter).clone()   #local id in block matrix

                    if len(in_sample_train_data_index) == 0:
                        continue
                    try:
                        adj[sample_index][:,sample_index]
                    except: #adj is empty
                        continue
                    
                    acc_train = FedSage_train(epoch, models[i], optimizers[i], 
                                                        features, adj, labels, sample_index, in_sample_train_data_index)
                acc_trains.append(acc_train)
                
            states=[]
            gloabl_state=dict()
            for i in range(K):
                states.append(models[i].state_dict())
            # Average all parameters
            for key in global_model.state_dict():
                gloabl_state[key] = in_com_train_data_indexes[0].shape[0] * states[0][key]
                count_D=in_com_train_data_indexes[0].shape[0]
                for i in range(1,K):
                    gloabl_state[key] += in_com_train_data_indexes[i].shape[0] * states[i][key]
                    count_D += in_com_train_data_indexes[i].shape[0]
                gloabl_state[key] /= count_D

            global_model.load_state_dict(gloabl_state)
            
            # Testing
            
            loss_train, acc_train = test(global_model, features, adj, labels, idx_train)
            print(t,'\t',"train",'\t',loss_train,'\t',acc_train)
            
            loss_val, acc_val = test(global_model, features, adj, labels, idx_val) #validation
            print(t,'\t',"val",'\t',loss_val,'\t',acc_val)
            

            a = open(dataset_name+'_IID_'+str(iid_percent)+'_' + str(L_hop) +'hop_BDS_federated_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations)+'_epoch_'+str(args_epochs)+'_device_num_'+str(K),'a+')

            
            a.write(str(t)+'\t'+"train"+'\t'+str(loss_train)+'\t'+str(acc_train)+'\n')
            a.write(str(t)+'\t'+"val"+'\t'+str(loss_val)+'\t'+str(acc_val)+'\n')
            a.close()
            for i in range(K):
                models[i].load_state_dict(gloabl_state)
        #test  
        loss_test, acc_test= test(global_model, features, adj, labels, idx_test)
        print(t,'\t',"test",'\t',loss_test,'\t',acc_test)
        a = open(dataset_name+'_IID_'+str(iid_percent)+'_' + str(L_hop) +'hop_BDS_federated_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations)+'_epoch_'+str(args_epochs)+'_device_num_'+str(K),'a+')
        a.write(str(t)+'\t'+"test"+'\t'+str(loss_test)+'\t'+str(acc_test)+'\n')
        a.close()
        print("save file as",dataset_name+'_IID_'+str(iid_percent)+'_' + str(L_hop) +'hop_BDS_federated_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations)+'_epoch_'+str(args_epochs)+'_device_num_'+str(K))
        
        del global_model
        del features 
        del adj
        del labels
        del idx_train
        del idx_val
        del idx_test
        while(len(models)>=1):
            del models[0]
        
        return loss_test, acc_test



In [9]:
def FedSage_plus(K, features, adj, labels, idx_train, idx_val, idx_test, iid_percent, L_hop = 1, num_layers=2):
        # K: number of models
        #choose adj matrix
        #multilayer_GCN:n*n
        #define model
        global_model = GCN(nfeat=features.shape[1],
                nhid=args_hidden,
                nclass=labels.max().item() + 1,
                dropout=args_dropout,
                NumLayers=num_layers)
        global_model.reset_parameters()
        models=[]
        for i in range(K):
            models.append(GCN(nfeat=features.shape[1],
                nhid=args_hidden,
                nclass=labels.max().item() + 1,
                dropout=args_dropout,
                NumLayers=num_layers))
        if args_cuda:
                for i in range(K):
                    models[i]=models[i].to(device)
                global_model=global_model.to(device)
                features = features.to(device)
                adj = adj.to(device)
                labels = labels.to(device)
                idx_train = idx_train.to(device)
                idx_val = idx_val.to(device)
                idx_test = idx_test.to(device)
        #optimizer and train
        optimizers=[]
        for i in range(K):
            optimizers.append(optim.SGD(models[i].parameters(),
                              lr=args_lr, weight_decay=args_weight_decay))
        # Train model
        
        row, col, edge_attr = adj.t().coo()
        edge_index = torch.stack([row, col], dim=0)
        
        
        split_data_indexes=[]
        
        nclass=labels.max().item() + 1
        split_data_indexes = []
        non_iid_percent = 1 - float(iid_percent)
        iid_indexes = [] #random assign
        shuffle_labels = [] #make train data points split into different devices
        for i in range(K):
            current = torch.nonzero(labels == i).reshape(-1)
            current = current[np.random.permutation(len(current))] #shuffle
            shuffle_labels.append(current)
                
        average_device_of_class = K // nclass
        if K % nclass != 0: #for non-iid
            average_device_of_class += 1
        for i in range(K):  
            label_i= i // average_device_of_class    
            labels_class = shuffle_labels[label_i]

            average_num= int(len(labels_class)//average_device_of_class * non_iid_percent)
            split_data_indexes.append((labels_class[average_num * (i % average_device_of_class):average_num * (i % average_device_of_class + 1)]))
        
        if args_cuda:
            iid_indexes = setdiff1d(torch.tensor(range(len(labels))).to(device), torch.cat(split_data_indexes))
        else:
            iid_indexes = setdiff1d(torch.tensor(range(len(labels))), torch.cat(split_data_indexes))
        
        iid_indexes = iid_indexes[np.random.permutation(len(iid_indexes))]
        
        for i in range(K):  #for iid
            label_i= i // average_device_of_class
            labels_class = shuffle_labels[label_i]

            average_num= int(len(labels_class)//average_device_of_class * (1 - non_iid_percent))
            split_data_indexes[i] = list(split_data_indexes[i]) + list(iid_indexes[:average_num])
                    
            iid_indexes = iid_indexes[average_num:]
            
        communicate_indexes = []
        in_com_train_data_indexes = []
        for i in range(K):
            if args_cuda:
                split_data_indexes[i] = torch.tensor(split_data_indexes[i]).to(device)
            else:
                split_data_indexes[i] = torch.tensor(split_data_indexes[i])
                
            split_data_indexes[i] = split_data_indexes[i].sort()[0]
            
            #communicate_index=get_K_hop_neighbors(adj, split_data_indexes[i], L_hop) #normalized adj
            
            communicate_index = torch_geometric.utils.k_hop_subgraph(split_data_indexes[i],L_hop,edge_index)[0]
                
            communicate_indexes.append(communicate_index)
            communicate_indexes[i] = communicate_indexes[i].sort()[0]
            
            inter = intersect1d(split_data_indexes[i], idx_train)  ###only count the train data of nodes in current server(not communicate nodes)

                
            in_com_train_data_indexes.append(torch.searchsorted(communicate_indexes[i], inter).clone())   #local id in block matrix
        
        features_in_clients = []
        #assume the linear generator learnt the optimal (the average of features of neighbor nodes)
        #gaussian noise
        for i in range(K):
            #orignial features of outside neighbors of nodes in client i
            original_feature_i = features[setdiff1d(split_data_indexes[i], communicate_indexes[i])].clone()
            
            gaussian_feature_i = original_feature_i + torch.normal(0, 0.1, original_feature_i.shape).to(device)
            
            copy_feature = features.clone()
            
            copy_feature[setdiff1d(split_data_indexes[i], communicate_indexes[i])] = gaussian_feature_i
            
            features_in_clients.append(copy_feature[communicate_indexes[i]])
            print(features_in_clients[i].shape, communicate_indexes[i].shape)

        #assign global model weights to local models at initial step
        for i in range(K):
            models[i].load_state_dict(global_model.state_dict())
        
        for t in range(args_iterations):
            acc_trains=[]
            for i in range(K):
                for epoch in range(args_epochs):
                    if len(in_com_train_data_indexes[i]) == 0:
                        continue
                    try:
                        adj[communicate_indexes[i]][:,communicate_indexes[i]]
                    except: #adj is empty
                        continue
                    acc_train = FedSage_train(epoch, models[i], optimizers[i], 
                                                        features_in_clients[i], adj, labels, communicate_indexes[i], in_com_train_data_indexes[i])
                    
                acc_trains.append(acc_train)
            states=[]
            gloabl_state=dict()
            for i in range(K):
                states.append(models[i].state_dict())
            # Average all parameters
            for key in global_model.state_dict():
                gloabl_state[key] = in_com_train_data_indexes[0].shape[0] * states[0][key]
                count_D=in_com_train_data_indexes[0].shape[0]
                for i in range(1,K):
                    gloabl_state[key] += in_com_train_data_indexes[i].shape[0] * states[i][key]
                    count_D += in_com_train_data_indexes[i].shape[0]
                gloabl_state[key] /= count_D

            global_model.load_state_dict(gloabl_state)
            
            # Testing
            
            loss_train, acc_train = test(global_model, features, adj, labels, idx_train)
            print(t,'\t',"train",'\t',loss_train,'\t',acc_train)
            
            loss_val, acc_val = test(global_model, features, adj, labels, idx_val) #validation
            print(t,'\t',"val",'\t',loss_val,'\t',acc_val)
            

            a = open(dataset_name+'_IID_'+str(iid_percent)+'_' + str(L_hop) +'hop_FedSage_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations)+'_epoch_'+str(args_epochs)+'_device_num_'+str(K),'a+')

            
            a.write(str(t)+'\t'+"train"+'\t'+str(loss_train)+'\t'+str(acc_train)+'\n')
            a.write(str(t)+'\t'+"val"+'\t'+str(loss_val)+'\t'+str(acc_val)+'\n')
            a.close()
            for i in range(K):
                models[i].load_state_dict(gloabl_state)
        #test  
        loss_test, acc_test= test(global_model, features, adj, labels, idx_test)
        print(t,'\t',"test",'\t',loss_test,'\t',acc_test)
        a = open(dataset_name+'_IID_'+str(iid_percent)+'_' + str(L_hop) +'hop_FedSage_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations)+'_epoch_'+str(args_epochs)+'_device_num_'+str(K),'a+')
        a.write(str(t)+'\t'+"test"+'\t'+str(loss_test)+'\t'+str(acc_test)+'\n')
        a.close()
        print("save file as",dataset_name+'_IID_'+str(iid_percent)+'_' + str(L_hop) +'hop_FedSage_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations)+'_epoch_'+str(args_epochs)+'_device_num_'+str(K))
        
        del global_model
        del features 
        del adj
        del labels
        del idx_train
        del idx_val
        del idx_test
        while(len(models)>=1):
            del models[0]
        
        return loss_test, acc_test



In [18]:
def Lhop_Block_federated_GCN(K, features, adj, labels, idx_train, idx_val, idx_test, iid_percent, L_hop, num_layers):
        # K: number of models
        K = int(K)
        #choose adj matrix
        #multilayer_GCN:n*n
        #define model
        global_model = GCN(nfeat=features.shape[1],
                nhid=args_hidden,
                nclass=labels.max().item() + 1,
                dropout=args_dropout,
                NumLayers=num_layers)
        global_model.reset_parameters()
        models=[]
        for i in range(K):
            models.append(GCN(nfeat=features.shape[1],
                nhid=args_hidden,
                nclass=labels.max().item() + 1,
                dropout=args_dropout,
                NumLayers=num_layers))
        if args_cuda:
                for i in range(K):
                    models[i]=models[i].cuda()
                global_model=global_model.cuda()
                features = features.cuda()
                adj = adj.cuda()
                labels = labels.cuda()
                idx_train = idx_train.cuda()
                idx_val = idx_val.cuda()
                idx_test = idx_test.cuda()
        #optimizer and train
        optimizers=[]
        for i in range(K):
            optimizers.append(optim.SGD(models[i].parameters(),
                              lr=args_lr, weight_decay=args_weight_decay))
        # Train model
        
        row, col, edge_attr = adj.t().coo()
        edge_index = torch.stack([row, col], dim=0)
        
        
        split_data_indexes=[]
        
        nclass=labels.max().item() + 1
        split_data_indexes = []
        non_iid_percent = 1 - float(iid_percent)
        iid_indexes = [] #random assign
        shuffle_labels = [] #make train data points split into different devices
        for i in range(K):
            current = torch.nonzero(labels == i).reshape(-1)
            current = current[np.random.permutation(len(current))] #shuffle
            shuffle_labels.append(current)
                
        average_device_of_class = K // nclass
        if K % nclass != 0: #for non-iid
            average_device_of_class += 1
        for i in range(K):  
            label_i= int(i // average_device_of_class)    
            labels_class = shuffle_labels[label_i]

            average_num= int(len(labels_class)//average_device_of_class * non_iid_percent)
            split_data_indexes.append((labels_class[int(average_num * (i % average_device_of_class)):int(average_num * (i % average_device_of_class + 1))]))
        
        if args_cuda:
            iid_indexes = setdiff1d(torch.tensor(range(len(labels))).cuda(), torch.cat(split_data_indexes))
        else:
            iid_indexes = setdiff1d(torch.tensor(range(len(labels))), torch.cat(split_data_indexes))
        iid_indexes = iid_indexes[np.random.permutation(len(iid_indexes))]
        
        for i in range(K):  #for iid
            label_i= int(i // average_device_of_class)
            labels_class = shuffle_labels[label_i]

            average_num= int(len(labels_class)//average_device_of_class * (1 - non_iid_percent))
            split_data_indexes[i] = list(split_data_indexes[i]) + list(iid_indexes[:average_num])
                    
            iid_indexes = iid_indexes[average_num:]
            
        communicate_indexes = []
        in_com_train_data_indexes = []
        for i in range(K):
            if args_cuda:
                split_data_indexes[i] = torch.tensor(split_data_indexes[i]).cuda()
            else:
                split_data_indexes[i] = torch.tensor(split_data_indexes[i])
                
            split_data_indexes[i] = split_data_indexes[i].sort()[0]
            
            #communicate_index=get_K_hop_neighbors(adj, split_data_indexes[i], L_hop) #normalized adj
            
            communicate_index = torch_geometric.utils.k_hop_subgraph(split_data_indexes[i],L_hop,edge_index)[0]
                
            communicate_indexes.append(communicate_index)
            communicate_indexes[i] = communicate_indexes[i].sort()[0]
            
            inter = intersect1d(split_data_indexes[i], idx_train)  ###only count the train data of nodes in current server(not communicate nodes)

                
            in_com_train_data_indexes.append(torch.searchsorted(communicate_indexes[i], inter).clone())   #local id in block matrix

        #assign global model weights to local models at initial step
        for i in range(K):
            models[i].load_state_dict(global_model.state_dict())
        
        for t in range(args_iterations):
            acc_trains=[]
            for i in range(K):
                for epoch in range(args_epochs):
                    if len(in_com_train_data_indexes[i]) == 0:
                        continue
                    
                    try:
                        adj[communicate_indexes[i]][:,communicate_indexes[i]]
                    except: #adj is empty
                        continue
                    acc_train = Lhop_Block_matrix_train(epoch, models[i], optimizers[i], 
                                                        features, adj, labels, communicate_indexes[i], in_com_train_data_indexes[i])
                    
                acc_trains.append(acc_train)
            states=[]
            gloabl_state=dict()
            for i in range(K):
                states.append(models[i].state_dict())
            # Average all parameters
            for key in global_model.state_dict():
                gloabl_state[key] = in_com_train_data_indexes[0].shape[0] * states[0][key]
                count_D=in_com_train_data_indexes[0].shape[0]
                for i in range(1,K):
                    gloabl_state[key] += in_com_train_data_indexes[i].shape[0] * states[i][key]
                    count_D += in_com_train_data_indexes[i].shape[0]
                gloabl_state[key] /= count_D

            global_model.load_state_dict(gloabl_state)
            
            # Testing
            
            loss_train, acc_train = test(global_model, features, adj, labels, idx_train)
            print(t,'\t',"train",'\t',loss_train,'\t',acc_train)
            
            loss_val, acc_val = test(global_model, features, adj, labels, idx_val) #validation
            print(t,'\t',"val",'\t',loss_val,'\t',acc_val)
            
            #evaluation metrics -> Precision, Recall, Balanced Accuracy, and F1-score. 
            precision_sc, recalL_sc, balanced_acc_sc, f1_test = eval_metrics(global_model, features, adj, labels, idx_test)
            print(f"{precision_sc}, {recalL_sc}, {balanced_acc_sc}, {f1_test}".format(precision_sc,recalL_sc,balanced_acc_sc,f1_test))

            a = open(dataset_name+'_IID_'+str(iid_percent)+'_' + str(L_hop) +'hop_Block_federated_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations)+'_epoch_'+str(args_epochs)+'_device_num_'+str(K),'a+')

            a.write(str(t)+'\t'+"train"+'\t'+str(loss_train)+'\t'+str(acc_train)+'\n')
            a.write(str(t)+'\t'+"val"+'\t'+str(loss_val)+'\t'+str(acc_val)+'\n')
            a.write(str(t)+'\t'+str(precision_sc) + '\t'+str(recalL_sc) +'\t' +str(balanced_acc_sc) +'\t' +str(f1_test)) 
        
            a.close()
            for i in range(K):
                models[i].load_state_dict(gloabl_state)
        #test  
        loss_test, acc_test= test(global_model, features, adj, labels, idx_test)
        print(t,'\t',"test",'\t',loss_test,'\t',acc_test)
        a = open(dataset_name+'_IID_'+str(iid_percent)+'_' + str(L_hop) +'hop_Block_federated_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations)+'_epoch_'+str(args_epochs)+'_device_num_'+str(K),'a+')
        a.write(str(t)+'\t'+"test"+'\t'+str(loss_test)+'\t'+str(acc_test)+'\n')
        a.close()
        print("save file as",dataset_name+'_IID_'+str(iid_percent)+'_' + str(L_hop) +'hop_Block_federated_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations)+'_epoch_'+str(args_epochs)+'_device_num_'+str(K))
        
        del global_model
        del features 
        del adj
        del labels
        del idx_train
        del idx_val
        del idx_test
        while(len(models)>=1):
            del models[0]
        
        return loss_test, acc_test



In [11]:
np.random.seed(42)
torch.manual_seed(42)
#'cora', 'citeseer', 'pubmed', 'amazon', #simulate, #'ogbn-arxiv' #other dataset twitter, 
dataset_name="amazon" 

if dataset_name == 'simulate':
    number_of_nodes=200
    class_num=3
    link_inclass_prob=10/number_of_nodes  #when calculation , remove the link in itself
    # EGCN good when network is dense 20/number_of_nodes  
    # #fails when network is sparse. 20/number_of_nodes/5

    link_outclass_prob=link_inclass_prob/20
    features, adj, labels, idx_train, idx_val, idx_test = generate_data(number_of_nodes,  class_num, link_inclass_prob, link_outclass_prob)               
else:
    features, adj, labels, idx_train, idx_val, idx_test = load_data_fraudre(dataset_name)
    class_num = labels.max().item() + 1

/home/shakib/Work/Implementation/FedGCN-3/Experiment_on_New_Dataset/data_process.py:291: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = nx.adjacency_matrix(nx.from_dict_of_lists(adj[0]))


In [12]:
if dataset_name in ['cora', 'citeseer', 'pubmed','amazon','simulate']:
    args_hidden = 16
else:
    args_hidden = 256

args_dropout = 0.5
args_lr = 1.0
args_weight_decay = 5e-4     #L2 penalty
args_epochs = 3
args_no_cuda = False
args_cuda = not args_no_cuda and torch.cuda.is_available()
# class_num = 3
args_device_num = class_num #split data into args_device_num parts
args_iterations = 500

In [13]:
#for testing
centralized_GCN(features, adj, labels, idx_train, idx_val, idx_test, num_layers = 2)

0 train 7372.38037109375 0.9050651230101302
0 val 7372.38037109375 0.9050651230101302
1 train 5836.75732421875 0.9050651230101302
1 val 5836.75732421875 0.9050651230101302
2 train 4301.90283203125 0.9050651230101302
2 val 4301.90283203125 0.9050651230101302
3 train 2767.81494140625 0.9050651230101302
3 val 2767.81494140625 0.9050651230101302
4 train 1234.4951171875 0.9050651230101302
4 val 1234.4951171875 0.9050651230101302
5 train 3155.939697265625 0.09493487698986976
5 val 3155.939697265625 0.09493487698986976
6 train 15961.6162109375 0.9050651230101302
6 val 15961.6162109375 0.9050651230101302
7 train 14421.697265625 0.9050651230101302
7 val 14421.697265625 0.9050651230101302
8 train 12882.55078125 0.9050651230101302
8 val 12882.55078125 0.9050651230101302
9 train 11344.173828125 0.9050651230101302
9 val 11344.173828125 0.9050651230101302
10 train 9806.5654296875 0.9050651230101302
10 val 9806.5654296875 0.9050651230101302
11 train 8269.7236328125 0.9050651230101302
11 val 8269.7236

(5953.63720703125, 0.9033564814814815)

In [14]:
for i in range(10):
    centralized_GCN(features, adj, labels, idx_train, idx_val, idx_test, num_layers = 2)

0 train 7365.42333984375 0.9050651230101302
0 val 7365.42333984375 0.9050651230101302
1 train 5829.8037109375 0.9050651230101302
1 val 5829.8037109375 0.9050651230101302
2 train 4294.95263671875 0.9050651230101302
2 val 4294.95263671875 0.9050651230101302
3 train 2760.86865234375 0.9050651230101302
3 val 2760.86865234375 0.9050651230101302
4 train 1227.5516357421875 0.9050651230101302
4 val 1227.5516357421875 0.9050651230101302
5 train 3228.68115234375 0.09493487698986976
5 val 3228.68115234375 0.09493487698986976
6 train 15954.6796875 0.9050651230101302
6 val 15954.6796875 0.9050651230101302
7 train 14414.765625 0.9050651230101302
7 val 14414.765625 0.9050651230101302
8 train 12875.623046875 0.9050651230101302
8 val 12875.623046875 0.9050651230101302
9 train 11337.2490234375 0.9050651230101302
9 val 11337.2490234375 0.9050651230101302
10 train 9799.64453125 0.9050651230101302
10 val 9799.64453125 0.9050651230101302
11 train 8262.8076171875 0.9050651230101302
11 val 8262.8076171875 0.9

/home/shakib/anaconda3/envs/cc/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


5 val 3325.979736328125 0.09493487698986976
6 train 15945.638671875 0.9050651230101302
6 val 15945.638671875 0.9050651230101302
7 train 14405.728515625 0.9050651230101302
7 val 14405.728515625 0.9050651230101302
8 train 12866.587890625 0.9050651230101302
8 val 12866.587890625 0.9050651230101302
9 train 11328.216796875 0.9050651230101302
9 val 11328.216796875 0.9050651230101302
10 train 9790.6171875 0.9050651230101302
10 val 9790.6171875 0.9050651230101302
11 train 8253.7861328125 0.9050651230101302
11 val 8253.7861328125 0.9050651230101302
12 train 6717.72216796875 0.9050651230101302
12 val 6717.72216796875 0.9050651230101302
13 train 5182.4267578125 0.9050651230101302
13 val 5182.4267578125 0.9050651230101302
14 train 3647.89892578125 0.9050651230101302
14 val 3647.89892578125 0.9050651230101302
15 train 2114.149169921875 0.9050651230101302
15 val 2114.149169921875 0.9050651230101302
16 train 581.9111938476562 0.8992764109985528
16 val 581.9111938476562 0.8992764109985528
17 train 100

In [ ]:
for args_epochs in [3]:
    for args_random_assign in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
        for i in range(10):
            Lhop_Block_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 0, num_layers = 2)
            #Lhop_Block_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 1, num_layers = 2)
            #Lhop_Block_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 2, num_layers = 2)

In [ ]:
for args_epochs in [3]:
    for args_random_assign in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
        for i in range(10):
            #Lhop_Block_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 0, num_layers = 2)
            Lhop_Block_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 1, num_layers = 2)
            #Lhop_Block_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 2, num_layers = 2)

In [ ]:
for args_epochs in [3]:
    for args_random_assign in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
        for i in range(10):
            #Lhop_Block_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 0, num_layers = 2)
            #Lhop_Block_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 1, num_layers = 2)
            Lhop_Block_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 2, num_layers = 2)

In [ ]:
for args_epochs in [3]:
    for args_random_assign in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
        for i in range(10):
            #Lhop_Block_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 0, num_layers = 2)
            #Lhop_Block_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 1, num_layers = 2)
            #Lhop_Block_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 2, num_layers = 2)
            #BDS_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign)


In [23]:
# for num_l in range(1, 11):
#     for i in range(10):#10 times
#         centralized_GCN(features, adj, labels, idx_train, idx_val, idx_test, num_layers = num_l)

In [24]:
# # 1hop_Block_federated_2layer_GCN_iter_500_epoch_3_device_num_7
# for i in range(10):
#     centralized_GCN(features, adj, labels, idx_train, idx_val, idx_test, num_layers = 2)
    
# for args_epochs in [3]:
#     for args_random_assign in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
#         for i in range(10):
#             #Lhop_Block_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 0, num_layers = 2)
#             #BDS_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign)
#             Lhop_Block_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 1, num_layers = 2)
#             #Lhop_Block_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 2, num_layers = 2)

In [25]:
# # 2hop_Block_federated_2layer_GCN_iter_500_epoch_3_device_num_7

# # for i in range(10):
# #     centralized_GCN(features, adj, labels, idx_train, idx_val, idx_test, num_layers = 2)
    
# for args_epochs in [3]:
#     for args_random_assign in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
#         for i in range(10):
#             #Lhop_Block_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 0, num_layers = 2)
#             #BDS_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign)
#             #Lhop_Block_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 1, num_layers = 2)
#             Lhop_Block_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 2, num_layers = 2)

In [26]:
# for args_epochs in [3]:
#     for args_random_assign in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
#         for num_l in range(2, 11):
#             for i in range(10):
#                 Block_federated_multilayer_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, num_layers = num_l)
#                 Lhop_Block_federated_multilayer_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 1, num_layers = num_l)
#                 Lhop_Block_federated_multilayer_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 2, num_layers = num_l)

END